<a href="https://colab.research.google.com/github/Ash-Daniels-Mo/Data-Structures-and-Algorithms/blob/main/Exercise_15_%26_16_(Hashtag).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm and Code Report: LRU Cache

## 1. Problem Statement

Design a data structure that implements a **Least Recently Used (LRU) cache** with the following capabilities:

1. `get(key)`: Return the value of the key if it exists in the cache; otherwise, return `-1`.
2. `put(key, value)`: Insert the key-value pair into the cache.  
   If the cache exceeds its capacity, it must **evict the least recently used key** before inserting the new item.

The cache must operate efficiently so that both `get` and `put` operations run in **$O(1)$ time complexity**.

---

## 2. Explanation of the Problem

An LRU cache stores a limited number of key-value pairs. When the cache reaches its capacity:

- The **least recently used** (oldest accessed) item is removed first.
- Newly accessed or inserted items become the **most recently used**.

The main challenges are:

1. Tracking the order of access so that the least recently used item can be efficiently removed.
2. Allowing `get` and `put` operations to run in $O(1)$ time.

A simple hash map alone is not sufficient because it does not track the order of usage. Similarly, a list or array alone would require $O(n)$ time for updating usage order. Hence, a combination of a **hash map** and a **doubly linked list** is commonly used.

---

## 3. Algorithm

The standard approach uses:

1. A **doubly linked list** to maintain the order of access:
   - Head points to the **most recently used** node.
   - Tail points to the **least recently used** node.
2. A **hash map** (dictionary) to store key-node pairs for $O(1)$ lookup.

Algorithm steps for `get(key)`:

1. Check if the key exists in the hash map.
2. If it does:
   - Move the corresponding node to the head of the linked list.
   - Return its value.
3. If it does not, return `-1`.

Algorithm steps for `put(key, value)`:

1. If the key exists:
   - Update its value.
   - Move the node to the head of the list.
2. If the key does not exist:
   - If the cache has reached its capacity:
     - Remove the tail node (least recently used).
     - Remove its key from the hash map.
   - Insert a new node at the head of the linked list.
   - Add the new key-node pair to the hash map.

This structure ensures **$O(1)$ time complexity** for both `get` and `put` operations.

---

## Time and Space Complexity

- **Time Complexity:**  
  $O(1)$ for both `get` and `put` operations, due to the combination of hash map and doubly linked list.

- **Space Complexity:**  
  $O(n)$, where $n$ is the capacity of the cache, to store the nodes and the hash map.


In [1]:
class Node:
    """
    Doubly linked list node used in the LRU cache.
    """
    def __init__(self, key, value):
        self.key = key          # Key of the node
        self.value = value      # Value of the node
        self.prev = None        # Pointer to the previous node
        self.next = None        # Pointer to the next node


class LRUCache:
    """
    LRU Cache implementation using a hash map and a doubly linked list.
    """
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # Key -> Node mapping

        # Dummy head and tail to avoid edge cases in insertion/deletion
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove(self, node: Node):
        """
        Removes a node from the doubly linked list.
        """
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node

    def _add_to_head(self, node: Node):
        """
        Adds a node right after the head (marks it as most recently used).
        """
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node

    def get(self, key: int) -> int:
        """
        Retrieves the value of the key if it exists, else returns -1.
        Moves the accessed node to the head (most recently used).
        """
        if key in self.cache:
            node = self.cache[key]
            self._remove(node)
            self._add_to_head(node)
            return node.value
        return -1

    def put(self, key: int, value: int) -> None:
        """
        Inserts or updates a key-value pair in the cache.
        If the cache exceeds its capacity, evicts the least recently used item.
        """
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self._remove(node)
            self._add_to_head(node)
        else:
            if len(self.cache) >= self.capacity:
                # Remove the least recently used node (before tail)
                lru_node = self.tail.prev
                self._remove(lru_node)
                del self.cache[lru_node.key]
            new_node = Node(key, value)
            self.cache[key] = new_node
            self._add_to_head(new_node)


# Example usage
cache = LRUCache(2)  # Capacity of 2

cache.put(1, 1)
cache.put(2, 2)
print(cache.get(1))    # Returns 1

cache.put(3, 3)        # Evicts key 2
print(cache.get(2))    # Returns -1 (not found)

cache.put(4, 4)        # Evicts key 1
print(cache.get(1))    # Returns -1 (not found)
print(cache.get(3))    # Returns 3
print(cache.get(4))    # Returns 4


1
-1
-1
3
4


# Algorithm and Code Report: First Missing Positive

## 1. Problem Statement

Given an unsorted integer array `nums`, the task is to return the **smallest missing positive integer**.

The solution must satisfy the following constraints:

- Time complexity: $O(n)$
- Space complexity: $O(1)$ (constant extra space)

---

## 2. Explanation of the Problem

The problem asks for the smallest positive integer that does **not appear** in the array.

For example:

```
nums = [1, 2, 0]
```

The smallest missing positive integer is `3`.

Another example:

```
nums = [3, 4, -1, 1]
```

The smallest missing positive integer is `2`.

A naive solution would sort the array and scan for the missing number, but sorting requires $O(n \log n)$ time and does not meet the space requirement.

The challenge is to find a way to track the presence of positive integers **within the array itself**, without using extra memory.

---

## 3. Algorithm

To solve this problem in $O(n)$ time and $O(1)$ extra space, we can use the array itself as a **hash map**:

1. Iterate through the array and **replace all non-positive numbers and numbers larger than $n$** with a placeholder (e.g., $n+1$), since they cannot be the smallest missing positive integer.
2. For each number `num` in the array:
   - If `1 ≤ |num| ≤ n`, mark the presence of `num` by **negating the value** at index `num - 1`.
3. After marking, iterate through the array:
   - The first index `i` with a positive value indicates that the number `i + 1` is missing.
4. If all positions are marked (all negative), the missing number is $n + 1$.

This algorithm works because:

- Only numbers in the range $[1, n]$ can be the first missing positive.
- Using **sign marking**, we can track presence without extra space.

---

## Time and Space Complexity

- **Time Complexity:**  
  $O(n)$, since each step involves a single pass through the array.

- **Space Complexity:**  
  $O(1)$, because the array itself is used for marking presence, without extra data structures.


In [2]:
def first_missing_positive(nums):
    """
    Finds the smallest missing positive integer in an unsorted array.

    Args:
        nums (list[int]): Unsorted integer array.

    Returns:
        int: The smallest missing positive integer.
    """

    n = len(nums)

    # Step 1: Replace non-positive numbers and numbers > n with a placeholder (n+1)
    for i in range(n):
        if nums[i] <= 0 or nums[i] > n:
            nums[i] = n + 1

    # Step 2: Use index as a hash key and mark presence by negating the value at that index
    for i in range(n):
        num = abs(nums[i])
        if 1 <= num <= n:
            # Negate the value at index num-1 if it's positive
            if nums[num - 1] > 0:
                nums[num - 1] = -nums[num - 1]

    # Step 3: Find the first index with a positive value
    for i in range(n):
        if nums[i] > 0:
            # i+1 is missing
            return i + 1

    # Step 4: If all numbers 1..n are present, the missing number is n+1
    return n + 1


# Example usage
nums = [3, 4, -1, 1]
print(first_missing_positive(nums))  # Output: 2

nums = [1, 2, 0]
print(first_missing_positive(nums))  # Output: 3


2
3
